In [2]:
%run read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-02c167b4-fd2c-4231-b6f4-d4b05ec6bc70;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 196ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/03/09 03:05:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 03:05:13 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/03/09 03:05:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [10]:
df_eps = readCheckpoint()

 3:12AM UTC on Mar 08, 2023 --- read and cache time: 0.4459545612335205 seconds ---


In [12]:
df_eps.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)



In [16]:
print(df_eps.count())
df_eps.select("object").distinct().show()
df_eps.select("event_day").distinct().show()
df_eps.select("event_hour").distinct().show()

10000000
+-------+
| object|
+-------+
|  SHELL|
|   FLOW|
|   FILE|
|PROCESS|
+-------+

+---------+
|event_day|
+---------+
|       23|
+---------+

+----------+
|event_hour|
+----------+
|        16|
|        15|
|        17|
|        18|
|        13|
|        14|
|        19|
+----------+



In [3]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline

# Create a PySxpark DataFrame
df = spark.createDataFrame([(0, "category1"), (1, "category2"), (2, "category3"), (3, "category1")], ["id", "category"])


In [4]:
df.toPandas()

,id,category
0,0,category1
1,1,category2
2,2,category3
3,3,category1


In [9]:
from pyspark.ml.feature import StringIndexer

#turn into numeric index before encoding
indexer = StringIndexer(inputCol='category', outputCol='class_numeric')
indexer_fitted = indexer.fit(df)
df_indexed = indexer_fitted.transform(df)
df_indexed.show()

encoder = OneHotEncoder(inputCols=['class_numeric'], outputCols=['class_onehot'],dropLast=False)
df_onehot = encoder.fit(df_indexed).transform(df_indexed)
df_onehot.show()

+---+---------+-------------+
| id| category|class_numeric|
+---+---------+-------------+
|  0|category1|          0.0|
|  1|category2|          1.0|
|  2|category3|          2.0|
|  3|category1|          0.0|
+---+---------+-------------+

+---+---------+-------------+-------------+
| id| category|class_numeric| class_onehot|
+---+---------+-------------+-------------+
|  0|category1|          0.0|(3,[0],[1.0])|
|  1|category2|          1.0|(3,[1],[1.0])|
|  2|category3|          2.0|(3,[2],[1.0])|
|  3|category1|          0.0|(3,[0],[1.0])|
+---+---------+-------------+-------------+



In [12]:
#now turn sparse array into columns
from pyspark.ml.functions import vector_to_array
df_col_onehot = df_onehot.select('*', vector_to_array('class_onehot').alias('col_onehot'))
df_col_onehot.show()

#and then unpack vectors into columns
num_categories = len(df_col_onehot.first()['col_onehot'])   # 3
cols_expanded = [(col('col_onehot')[i]) for i in range(num_categories)]
df_cols_onehot = df_col_onehot.select('id', 'category', *cols_expanded)
df_cols_onehot.show()

+---+---------+-------------+-------------+---------------+
| id| category|class_numeric| class_onehot|     col_onehot|
+---+---------+-------------+-------------+---------------+
|  0|category1|          0.0|(3,[0],[1.0])|[1.0, 0.0, 0.0]|
|  1|category2|          1.0|(3,[1],[1.0])|[0.0, 1.0, 0.0]|
|  2|category3|          2.0|(3,[2],[1.0])|[0.0, 0.0, 1.0]|
|  3|category1|          0.0|(3,[0],[1.0])|[1.0, 0.0, 0.0]|
+---+---------+-------------+-------------+---------------+

+---+---------+-------------+-------------+-------------+
| id| category|col_onehot[0]|col_onehot[1]|col_onehot[2]|
+---+---------+-------------+-------------+-------------+
|  0|category1|          1.0|          0.0|          0.0|
|  1|category2|          0.0|          1.0|          0.0|
|  2|category3|          0.0|          0.0|          1.0|
|  3|category1|          1.0|          0.0|          0.0|
+---+---------+-------------+-------------+-------------+



In [13]:
#now rename the categories
cols_expanded = [col('col_onehot')[i].alias(f'{indexer_fitted.labels[i]}') for i in range(num_categories)]
df_cols_onehot = df_col_onehot.select('id', 'category', *cols_expanded)
df_cols_onehot.show()

+---+---------+---------+---------+---------+
| id| category|category1|category2|category3|
+---+---------+---------+---------+---------+
|  0|category1|      1.0|      0.0|      0.0|
|  1|category2|      0.0|      1.0|      0.0|
|  2|category3|      0.0|      0.0|      1.0|
|  3|category1|      1.0|      0.0|      0.0|
+---+---------+---------+---------+---------+



In [15]:
from pyspark.ml.feature import BucketedRandomProjectionLSH, VectorAssembler

#now create an index based on the sparse vector
# create a BucketedRandomProjectionLSH instance to hash the sparse vector column
brp = BucketedRandomProjectionLSH(inputCol="class_onehot", outputCol="HashedFeatures", numHashTables=3, bucketLength=0.1)
# fit the transformer on the DataFrame and transform it
brp_model = brp.fit(df_onehot)
df_hashed = brp_model.transform(df_onehot)
# show the resulting DataFrame
df_hashed.show()


+---+---------+-------------+-------------+--------------------+
| id| category|class_numeric| class_onehot|      HashedFeatures|
+---+---------+-------------+-------------+--------------------+
|  0|category1|          0.0|(3,[0],[1.0])|[[1.0], [-4.0], [...|
|  1|category2|          1.0|(3,[1],[1.0])|[[8.0], [-7.0], [...|
|  2|category3|          2.0|(3,[2],[1.0])|[[4.0], [6.0], [-...|
|  3|category1|          0.0|(3,[0],[1.0])|[[1.0], [-4.0], [...|
+---+---------+-------------+-------------+--------------------+



In [17]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.linalg import Vectors

# create a VectorIndexer instance to assign integers to the unique hashed values
vi = VectorIndexer(inputCol="HashedFeatures", outputCol="IndexedFeatures", maxCategories=3)

# transform the DataFrame using the VectorIndexer
df_indexed = vi.fit(df_hashed).transform(df_hashed)

# show the resulting DataFrame
df_indexed.show()

IllegalArgumentException: requirement failed: Column HashedFeatures must be of type class org.apache.spark.ml.linalg.VectorUDT:struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually class org.apache.spark.sql.types.ArrayType:array<struct<type:tinyint,size:int,indices:array<int>,values:array<double>>>.

In [20]:
df_hashed = df_hashed.withColumn('color',lit(['r','g','r','r']))

Py4JJavaError: An error occurred while calling z:org.apache.spark.sql.functions.lit.
: org.apache.spark.SparkRuntimeException: The feature is not supported: literal for '[r, g, r, r]' of class java.util.ArrayList.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.literalTypeUnsupportedError(QueryExecutionErrors.scala:283)
	at org.apache.spark.sql.catalyst.expressions.Literal$.apply(literals.scala:101)
	at org.apache.spark.sql.functions$.lit(functions.scala:125)
	at org.apache.spark.sql.functions.lit(functions.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4

In [40]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.linalg import Vectors

# create sample data
data = [(0, "red",'g'), (1, "green",'blue'), (2, "red",'g')]

# create dataframe
df = spark.createDataFrame(data, ["id", "color",'second_color'])

df.printSchema()

#now create a hash
df = df.withColumn("sha2",sha2(concat_ws('||',col('color'),col('second_color')),256))

df.printSchema()




root
 |-- id: long (nullable = true)
 |-- color: string (nullable = true)
 |-- second_color: string (nullable = true)

root
 |-- id: long (nullable = true)
 |-- color: string (nullable = true)
 |-- second_color: string (nullable = true)
 |-- sha2: string (nullable = true)



In [41]:
#turn into numeric index before encoding
indexer = StringIndexer(inputCol='sha2', outputCol='class_numeric')
indexer_fitted = indexer.fit(df)
df_indexed = indexer_fitted.transform(df)
df_indexed.show()

+---+-----+------------+--------------------+-------------+
| id|color|second_color|                sha2|class_numeric|
+---+-----+------------+--------------------+-------------+
|  0|  red|           g|1edcfa2380771ecfb...|          0.0|
|  1|green|        blue|50e3429807908a500...|          1.0|
|  2|  red|           g|1edcfa2380771ecfb...|          0.0|
+---+-----+------------+--------------------+-------------+



In [ ]:
#create binary columns and then drop categorical column